### CMPS242 Homework 5 - Classifying Donald Trump and Hillary Clinton's Tweets
#### Team Members:
Olexiy Burov                             
Karthik Balakrishna           
Roy Shadmon     
Sasidharan Mahalingam


#### Importing the required packages
First import the rerquired packages required to create a neural network

In [1]:
#Importing Packages
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from tensorflow.contrib.rnn import RNNCell, LSTMStateTuple
import gensim as gs
import nltk 
nltk.download('punkt')

#Clearing any previous graph if present
tf.reset_default_graph()

[nltk_data] Downloading package punkt to /home/sasidharan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Loading Train data
Import the tweets from the given dataset

In [2]:
#TODO: Have to implement the text manupilation to remove the https and other unwanted characters and strings

#Loading the train dataset
#df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
df1 = pd.read_csv('train.csv')
#Loading the test dataset
#df2 = pd.read_csv('TrumpHillaryTestWithTags.csv')
df2 = pd.read_csv('test.csv')
#Getting only the tweets from the train dataset
all_tweets = df1['tweet'].str.replace("https://.*", "<url>").tolist()
#all_tweets = all_tweets + c_tweets
print(len(all_tweets))
#Doing a 70 - 30 split to form the train and test dataset
train_tweets = all_tweets[0:1000]
test_tweets = all_tweets[10000:11444]
#Printing a sample of the train dataset tweets
print('Printing a sample of the train dataset tweets... \n\n')
print(train_tweets[0:5])
#Printing a sample of the test dataset tweets
print('Printing a sample of the test dataset tweets... \n\n')
print(test_tweets[0:5])
#Concatenating the train and test dataset tweets to form the word embeddings
test_tweets2 = df2['tweet'].str.replace("https://.*", "<url>").tolist()
tweets = all_tweets + test_tweets2
#print(df1['tweet'].str.replace("https://*", "<url>"))

11444
Printing a sample of the train dataset tweets... 


['In Tampa, Florida- thank you to all of our outstanding volunteers who want to #MakeAmericaGreatAgain! <url>', 'Poll: @realDonaldTrump vs. @HillaryClinton among white Evangelicals. <url>', 'Obama on whether Trump could be trusted with US nuclear weapons: "Make your own judgment" <url>', '"Hillary Clinton has never quit on anything in her life." —@FLOTUS #DemsInPhilly <url>', 'I LOVE NEW YORK! #NewYorkValues \n<url>']
Printing a sample of the test dataset tweets... 


['“Donald Trump—The Disrupter” will air on @FoxNews Saturday night and Sunday night at 8 PM ET. Anchored by @BretBaier. @johnrobertsFox', 'Nobody beats me on National Security. \n<url>', 'Looks like I was right about NATO. I had no doubt. <url>', 'Jennifer is a terrific person. <url>', '"@R_U_OK_UK: @realDonaldTrump @glozee1 @PaulManafort @CNN @DanScavino Vote trump to save the west. Don\'t become like Europe - #WakeUpAmerica']


#### Tokenizing the tweets
Tokenize the tweets using keras Tokenizer

In [3]:
#Tokenize all the tweets
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(tweets)
#Printing some of the word indexs
sample_mapping = {k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]} 
print('Printing a few word index mappings... \n\n')
print(sample_mapping)


Printing a few word index mappings... 


{'luxury': 3485, "muted—there's": 6923, '“looks': 5771, 'compañeros': 8543, 'tapes': 8276, '51': 4294, 'que': 391, 'selfishly': 6620, 'sweat': 2361, 'out': 63}


#### Loading GloVe word embeddings
Load the downloaded pre-trained GloVe word vectors

In [4]:
#Setting the dimension of the word vectors

#Note: I have used 200 dimensional vectors (I know it might overfit as the given data is very less). 
#Let's zero down on the optimal value after we get this to work

embedding_dimension = 5
embed_method = "glove"
#Getting the word index representation of the tokenized words
word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}
#print(index_word)


#Create an empty dictionary that stores the word to index mappings
if(embed_method == "gensim"):
    embeddings_index = {}
    #Import the GloVe word vectors
    glove_data = 'glove.twitter.27B.200d.txt'
    #Load the word vectors into memory
    f = open(glove_data)
    for line in f:
        values = line.split()
        word = values[0]
        value = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = value
    f.close()

    #Printing the number of word vectors loaded
    print('Loaded %s word vectors.' % len(embeddings_index))
    


    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimension), dtype=np.float64)
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector[:embedding_dimension]

    
else:
    tokens = tokenizer.texts_to_sequences(tweets)
    sentences = []
    for sentence in tokens:
        s = []
        for index in sentence:
            word = index_word[index]
            s.append(word)
        sentences.append(s)
    #print(sentences)
    model = gs.models.Word2Vec(sentences , min_count=1, 
                                          size = embedding_dimension, iter = 5000)
    #model = gs.models.KeyedVectors.load_word2vec_format('glove.27B.200d.w2vformat.txt', binary=False)
    #model.train(sentences , min_count=1, size = embedding_dimension, epochs = 1000)
    embedding_matrix = np.zeros((len(model.wv.vocab) + 1, embedding_dimension),dtype=np.float32)
    for i in range(len(model.wv.vocab)):
        embedding_vector = model.wv[model.wv.index2word[i]]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
#Printing the shape of the word embedding matrix
print(embedding_matrix.shape)
print(type(embedding_matrix))
zero_rows = np.where(~embedding_matrix.any(axis=1))[0]
print(zero_rows.shape)

(10186, 5)
<class 'numpy.ndarray'>
(1,)


#### Get the tweets from the training dataset and find word embedding matrix
Get the training tweet dataset and convert that from strings into the word embedding matrix, the required input to the LSTM

In [5]:
#reshaping the training data to word to index sequences
train_X = tokenizer.texts_to_sequences(train_tweets)
#pad the shorter sentences to the length of the largest sentence
train_X = tf.keras.preprocessing.sequence.pad_sequences(train_X, maxlen=32)
#print the shape of the matrix of word vectors
print(train_X.shape)
print(type(train_X))

(1000, 32)
<class 'numpy.ndarray'>


#### Get the tweets from the test dataset and find the word embedding matrix for it
Get the test tweets and find the word embedding for the tweets

In [6]:
#reshaping the test data to word to index sequences
test_X = tokenizer.texts_to_sequences(test_tweets)
#pad the shorter sentences to the length of the largest sentence
test_X = tf.keras.preprocessing.sequence.pad_sequences(test_X, maxlen=32)
#print the shape of the matrix of word vectors
print(test_X.shape)
print(type(test_X))

#reshaping the test data to word to index sequences
test_X2 = tokenizer.texts_to_sequences(test_tweets2)
#pad the shorter sentences to the length of the largest sentence
test_X2 = tf.keras.preprocessing.sequence.pad_sequences(test_X2, maxlen=32)
#print the shape of the matrix of word vectors
print(test_X2.shape)
print(type(test_X2))

(1444, 32)
<class 'numpy.ndarray'>
(1444, 32)
<class 'numpy.ndarray'>


#### Define different parameters for the LSTM
Now we define the number of time steps, number of hidden layers and the number of nodes in each layer

In [7]:
#Set number of time steps
time_steps = 32
#Batch size
batch_size = 100
#Number of hidden layers
num_units = 5
#Learning rate
learning_rate = 0.001
#Number of classes
n_classes = 2
#Size of the word vectors
n_input = embedding_dimension
#Regularizer parameter
lambda_l2 = 0.001

#### Creating labels for train and test data set
Next, we form the train and test labels from the loaded dataframe

In [8]:
#Creating an empty list to store the labels
l1 = []
#Get the train labels
_ = df1['handle'].apply(lambda x: l1.append([1,0]) if x == 'realDonaldTrump' else l1.append([0,1]))
#Convert the train label list into an numpy array
all_labels = np.array(l1, dtype=np.float32)
train_labels = all_labels[0:10000]
print(train_labels.shape)
#Empty the list to store the test labels
#l1 = []
#Populate the list with test labels
#_ = df2['handle'].apply(lambda x: l1.append([1,0]) if x == 'realDonaldTrump' else l1.append([0,1]))
#Convert the test label list into an numpy array
#test_labels = np.array(l1, dtype=np.float64)
test_labels = all_labels[10000:11444]
print(test_labels.shape)

(10000, 2)
(1444, 2)


#### LSTM cell with Batch Normalization

In [9]:
def orthogonal_initializer(scale=1.0):
    def _initializer(shape, dtype=tf.float32, partition_info=None):
        if partition_info is not None:
            ValueError(
                "Do not know what to do with partition_info in BN_LSTMCell")
        flat_shape = (shape[0], np.prod(shape[1:]))
        a = np.random.normal(0.0, 1.0, flat_shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v
        q = q.reshape(shape)
        return tf.constant(scale * q[:shape[0], :shape[1]], dtype=dtype)
    return _initializer


def batch_norm(inputs, name_scope, is_training, epsilon=1e-3, decay=0.99):
    with tf.variable_scope(name_scope):
        size = inputs.get_shape().as_list()[1]

        scale = tf.get_variable(
            'scale', [size], initializer=tf.constant_initializer(0.1))
        offset = tf.get_variable('offset', [size])

        population_mean = tf.get_variable(
            'population_mean', [size],
            initializer=tf.zeros_initializer(), trainable=False)
        population_var = tf.get_variable(
            'population_var', [size],
            initializer=tf.ones_initializer(), trainable=False)
        batch_mean, batch_var = tf.nn.moments(inputs, [0])

        # The following part is based on the implementation of :
        # https://github.com/cooijmanstim/recurrent-batch-normalization
        train_mean_op = tf.assign(
            population_mean,
            population_mean * decay + batch_mean * (1 - decay))
        train_var_op = tf.assign(
            population_var, population_var * decay + batch_var * (1 - decay))

        if is_training is True:
            with tf.control_dependencies([train_mean_op, train_var_op]):
                return tf.nn.batch_normalization(
                    inputs, batch_mean, batch_var, offset, scale, epsilon)
        else:
            return tf.nn.batch_normalization(
                inputs, population_mean, population_var, offset, scale,
                epsilon)



class BN_LSTMCell(RNNCell):
    """LSTM cell with Recurrent Batch Normalization.
    This implementation is based on:
         http://arxiv.org/abs/1603.09025
    This implementation is also based on:
         https://github.com/OlavHN/bnlstm
         https://github.com/nicolas-ivanov/Seq2Seq_Upgrade_TensorFlow
    """

    def __init__(self, num_units, is_training,
                 use_peepholes=False, cell_clip=None,
                 initializer=orthogonal_initializer(),
                 num_proj=None, proj_clip=None,
                 forget_bias=1.0,
                 state_is_tuple=True,
                 activation=tf.tanh):
        """Initialize the parameters for an LSTM cell.
        Args:
          num_units: int, The number of units in the LSTM cell.
          is_training: bool, set True when training.
          use_peepholes: bool, set True to enable diagonal/peephole
            connections.
          cell_clip: (optional) A float value, if provided the cell state
            is clipped by this value prior to the cell output activation.
          initializer: (optional) The initializer to use for the weight
            matrices.
          num_proj: (optional) int, The output dimensionality for
            the projection matrices.  If None, no projection is performed.
          forget_bias: Biases of the forget gate are initialized by default
            to 1 in order to reduce the scale of forgetting at the beginning of
            the training.
          state_is_tuple: If True, accepted and returned states are 2-tuples of
            the `c_state` and `m_state`.  If False, they are concatenated
            along the column axis.
          activation: Activation function of the inner states.
        """
        if not state_is_tuple:
            tf.logging.log_first_n(
                tf.logging.WARN,
                "%s: Using a concatenated state is slower and "
                " will soon be deprecated.  Use state_is_tuple=True.", 1, self)

        self.num_units = num_units
        self.is_training = is_training
        self.use_peepholes = use_peepholes
        self.cell_clip = cell_clip
        self.num_proj = num_proj
        self.proj_clip = proj_clip
        self.initializer = initializer
        self.forget_bias = forget_bias
        self._state_is_tuple = state_is_tuple
        self.activation = activation

        if num_proj:
            self._state_size = (
                LSTMStateTuple(num_units, num_proj)
                if state_is_tuple else num_units + num_proj)
            self._output_size = num_proj
        else:
            self._state_size = (
                LSTMStateTuple(num_units, num_units)
                if state_is_tuple else 2 * num_units)
            self._output_size = num_units

    @property
    def state_size(self):
        return self._state_size

    @property
    def output_size(self):
        return self._output_size

    def __call__(self, inputs, state, scope=None):

        num_proj = self.num_units if self.num_proj is None else self.num_proj

        if self._state_is_tuple:
            (c_prev, h_prev) = state
        else:
            c_prev = tf.slice(state, [0, 0], [-1, self.num_units])
            h_prev = tf.slice(state, [0, self.num_units], [-1, num_proj])

        dtype = inputs.dtype
        input_size = inputs.get_shape().with_rank(2)[1]

        with tf.variable_scope(scope or type(self).__name__):
            if input_size.value is None:
                raise ValueError(
                    "Could not infer input size from inputs.get_shape()[-1]")

            W_xh = tf.get_variable(
                'W_xh',
                [input_size, 4 * self.num_units],
                initializer=self.initializer)
            W_hh = tf.get_variable(
                'W_hh',
                [num_proj, 4 * self.num_units],
                initializer=self.initializer)
            bias = tf.get_variable('B', [4 * self.num_units])

            xh = tf.matmul(inputs, W_xh)
            hh = tf.matmul(h_prev, W_hh)

            bn_xh = batch_norm(xh, 'xh', self.is_training)
            bn_hh = batch_norm(hh, 'hh', self.is_training)

            # i:input gate, j:new input, f:forget gate, o:output gate
            lstm_matrix = tf.nn.bias_add(tf.add(bn_xh, bn_hh), bias)
            i, j, f, o = tf.split(
                value=lstm_matrix, num_or_size_splits=4, axis=1)

            # Diagonal connections
            if self.use_peepholes:
                w_f_diag = tf.get_variable(
                    "W_F_diag", shape=[self.num_units], dtype=dtype)
                w_i_diag = tf.get_variable(
                    "W_I_diag", shape=[self.num_units], dtype=dtype)
                w_o_diag = tf.get_variable(
                    "W_O_diag", shape=[self.num_units], dtype=dtype)

            if self.use_peepholes:
                c = c_prev * tf.sigmoid(f + self.forget_bias +
                                        w_f_diag * c_prev) + \
                    tf.sigmoid(i + w_i_diag * c_prev) * self.activation(j)
            else:
                c = c_prev * tf.sigmoid(f + self.forget_bias) + \
                    tf.sigmoid(i) * self.activation(j)

            if self.cell_clip is not None:
                c = tf.clip_by_value(c, -self.cell_clip, self.cell_clip)

            bn_c = batch_norm(c, 'cell', self.is_training)

            if self.use_peepholes:
                h = tf.sigmoid(o + w_o_diag * c) * self.activation(bn_c)
            else:
                h = tf.sigmoid(o) * self.activation(bn_c)

            if self.num_proj is not None:
                w_proj = tf.get_variable(
                    "W_P", [self.num_units, num_proj], dtype=dtype)

                h = tf.matmul(h, w_proj)
                if self.proj_clip is not None:
                    h = tf.clip_by_value(h, -self.proj_clip, self.proj_clip)

            new_state = (LSTMStateTuple(c, h)
                         if self._state_is_tuple else tf.concat(1, [c, h]))

        return h, new_state

#### Declaring the weeights and biases for the final fed forward network and placeholders for the input data
Declaring variables for the weights and biases for the final feed forward network and the placeholders for the input dataset

In [10]:
#Creating a variable to hold the weights of the final feed forward layer
out_weights = tf.Variable(tf.random_normal([num_units, n_classes], dtype=tf.float32), dtype=tf.float32)
#Creating a variable to hold the biases of the final feed forward layer
out_bias = tf.Variable(tf.random_normal([n_classes],dtype=tf.float32),dtype=tf.float32)

#Input data placeholder
#x = tf.placeholder("float64",[None,time_steps,n_input])
x = tf.Variable(tf.zeros([train_X.shape[0],time_steps,n_input], dtype=tf.float32),dtype=tf.float32)
#x = tf.identity(embed)
#Input label placeholder
y = tf.placeholder("float32",[None,n_classes])
#y = tf.identity(train_labels)
X = tf.placeholder("int32", [None,32])
word_vecs = tf.placeholder("float32",[embedding_matrix.shape[0], n_input])
train_flag = tf.placeholder(tf.bool)


#### Defining the LSTM model
Then, define and construct the LSTM model

In [11]:
#tf.reset_default_graph()
#Copying values into placeholders
x = tf.nn.embedding_lookup(word_vecs, X)

#Reshaping the input dataset
input_data = tf.unstack(x, time_steps, 1)

#defining the network
#lstm_layer= tf.contrib.rnn.LSTMCell(num_units,forget_bias=1)
lstm_layer = BN_LSTMCell(num_units=num_units, is_training=train_flag, forget_bias=1, use_peepholes=True)
lstm_layer = tf.nn.rnn_cell.DropoutWrapper(lstm_layer, output_keep_prob=0.5)
#lstm_layer = tf.contrib.rnn.LayerNormBasicLSTMCell(num_units, dropout_keep_prob=0.5) 
outputs,_= tf.contrib.rnn.static_rnn(lstm_layer,input_data,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#defining regularization loss
#reg_loss = lambda_l2 * sum( tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
#loss += reg_loss
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#test prediction
test = tf.nn.softmax(prediction)

#### Running the LSTM model
Now that we have set up the LSTM model, we run the LSTM model

In [12]:
def generate_samples(in_data, in_labels, no_of_samples):
    indices = np.random.choice(in_data.shape[0],no_of_samples,replace=False)
    return(in_data[indices], in_labels[indices])


#initialize variables
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<10000:
        batch_x, batch_y = generate_samples(train_X, train_labels, batch_size)
        #print(batch_x.shape)
        #batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y})
        
        if iter %1000 == 0:
            acc = sess.run(accuracy,feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y})
            los = sess.run(loss,feed_dict={word_vecs:embedding_matrix, X: batch_x, y: batch_y, train_flag: True})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    output = sess.run(test, feed_dict={word_vecs:embedding_matrix, X: test_X2, y: test_labels, train_flag: False})
    np.savetxt("prediction.csv", output, delimiter=",")
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={word_vecs:embedding_matrix, X: test_X, y: test_labels,
                                                            train_flag: False}))
    print(output[0:10])

For iter  1000
Accuracy  0.62
Loss  0.641231
__________________
For iter  2000
Accuracy  0.7
Loss  0.579099
__________________
For iter  3000
Accuracy  0.77
Loss  0.562348
__________________
For iter  4000
Accuracy  0.76
Loss  0.533054
__________________
For iter  5000
Accuracy  0.75
Loss  0.536147
__________________
For iter  6000
Accuracy  0.82
Loss  0.500232
__________________
For iter  7000
Accuracy  0.74
Loss  0.527343
__________________
For iter  8000
Accuracy  0.73
Loss  0.538795
__________________
For iter  9000
Accuracy  0.79
Loss  0.423148
__________________
Testing Accuracy: 0.722992
[[ 0.11456414  0.88543588]
 [ 0.27194732  0.72805262]
 [ 0.86403149  0.13596851]
 [ 0.29079717  0.70920277]
 [ 0.33592841  0.66407156]
 [ 0.5874179   0.4125821 ]
 [ 0.6783812   0.32161885]
 [ 0.50130862  0.49869132]
 [ 0.54860324  0.45139676]
 [ 0.02242926  0.97757077]]
